# Import des modules 

In [15]:
import os
import requests
import zipfile
import csv
import time
import re
import pandas as pd
import os
import glob
import yaml


# Initialisation des chemins 

- Vous indiquez le chemin du dossier dans lequel vous souhaitez travailler 



In [4]:
#identification chemin
path_2 = r"C:\Users\ezano\Documents\GitHub\georisques_data"

## Liste des départements en France métropolitaine

- En fonction de la présentation des fichiers à télécharger sur le site, vous choisirez l'une des 2 listes ci-dessous

In [5]:
liste_departements = [
    "01", "02", "03", "04", "05", "06", "07", "08", "09", "10",
    "11", "12", "13", "14", "15", "16", "17", "18", "19", "20",
    "21", "22", "23", "24", "25", "26", "27", "28", "29", "30",
    "31", "32", "33", "34", "35", "36", "37", "38", "39", "40",
    "41", "42", "43", "44", "45", "46", "47", "48", "49", "50",
    "51", "52", "53", "54", "55", "56", "57", "58", "59", "60",
    "61", "62", "63", "64", "65", "66", "67", "68", "69", "70",
    "71", "72", "73", "74", "75", "76", "77", "78", "79", "80",
    "81", "82", "83", "84", "85", "86", "87", "88", "89", "90",
    "91", "92", "93", "94", "95","971", "972", "973", "974", "976"
]

annees = ["2019","2020","2021","2022","2023"]


##  Liens fichiers csv, zip et excel à télécharger 
- ici on initialise les liens des fichiers sur le site et on indique également  le type de liste à donner en paramètre à la fonction ci- après 

In [6]:
# Pour les liens ci-dessous, utiliser la liste_departements

url_inondation = "https://files.georisques.fr/di_2020/tri_2020_sig_di_{0}.zip"
url_eolien ="https://files.georisques.fr/bdeole/production/csv/eolien_dept{0}.zip"
url_inondation_par_remontee_nappes = "https://files.georisques.fr/REMNAPPES/Dept_{0}.zip"
inventaire_de_cavite = "https://www.georisques.gouv.fr/webappReport/ws/cavites/departements/{0}/fiches.csv?"
mvt_terrain = "https://www.georisques.gouv.fr/webappReport/ws/mvmt/departements/{0}/fichecommunes.csv?"
obligation_legale_debrousaillement = "https://files.georisques.fr/old_feu/{0}.zip"
zonage_inondation_2013 = "https://files.georisques.fr/di/TRI_{0}_SIG_DI.zip"
zonage_inondation_2020 = "https://files.georisques.fr/di_2020/tri_2020_sig_di_{0}.zip"
retrait_gonflement_argile = "https://files.georisques.fr/argiles/AleaRG{0}_L93.zip"
secteurs_info_sols = "https://mapsref.brgm.fr/wxs/georisques/georisques_dl?&service=wfs&version=2.0.0&request=getfeature&typename=classification&outputformat=csvtext&Filter=%3CFilter%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Ecode_dpt%3C/PropertyName%3E%3CLiteral%3E{0}%3C/Literal%3E%3C/PropertyIsEqualTo%3E%3C/Filter%3E"# dep
secteurs_sols = "https://mapsref.brgm.fr/wxs/georisques/georisques_dl?&service=wfs&version=2.0.0&request=getfeature&typename=classification&outputformat=csvtext&Filter=%3CFilter%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Ecode_dpt%3C/PropertyName%3E%3CLiteral%3E{0}%3C/Literal%3E%3C/PropertyIsEqualTo%3E%3C/Filter%3E"
site_et_sols_pollues = "https://www.georisques.gouv.fr/webappReport/ws/infosols/export/excel?codeDepartement={0}"

# Pour les liens ci-dessous, utiliser la liste_annee
installation_industrielle_polluant = "https://files.georisques.fr/irep/{0}.zip"

# télécharger manuellement car un seul fichier
procedures_risque = "https://files.georisques.fr/GASPAR/gaspar.zip"

# Application : Téléchargement, extraction et enregistrement des fichiers 
 
**Cette fonction est valable pour tous les liens ci-dessus**

## Note explicative : 
- la fonction téléchargement créee 2  sous- dossiers (téléchargement et extraction):
        -le dossier téléchargement : il contient tous les dossiers au format zip
        -le dossier extraction : il contien tous les dossiers dézippés (au format csv, shp etc.... selon le lien fourni) 
- teléchargement(liste_dep, url, nom_dossier) : 
        - url : on attribue à la fonction l'un des liens énumérés ci-dessus 
        -liste_dep : on attribue à la fonction la liste qui correspond au lien
        - nom_dossier : on nomme le sous-dossier dans lequel, on aimerait que le fichier soit 
              - Exemple d'utilisation 
                    - liste_dep = [ "02", "03"]  
                    - nom_dossier = "secteurs_info_sols"
                    - url = secteurs_info_sols
                    **teléchargement(liste_dep,secteurs_info_sols,nom_dossier)**
                    
NB : les documents seront enregistrés dans le dossier dans lequel se trouve le notebook


In [7]:
def teléchargement(liste_dep, url, nom_dossier,path_2):
    # proxies 
    proxies = {
    'http': "http://inetproxy:83",
    'https': "http://inetproxy:83"
    }
  
    path_ = os.getcwd()
    dossier_telechargement = os.path.join(path_2, fr"telechargements\{nom_dossier}")
    os.makedirs(dossier_telechargement, exist_ok=True)

    for dep_ in liste_dep:
        try:
           
            url_ = url.format(dep_)
            dossier_extraction = os.path.join(path_2, fr"{nom_dossier}\{dep_}")
            os.makedirs(dossier_extraction, exist_ok=True)
          
            nom_fichier = os.path.basename(url_)

            if "csv" in url_:
                chemin_csv_local = os.path.join(dossier_extraction, f"fichier{dep_}.csv")
                reponse = requests.get(url_,verify=False)
                if reponse.status_code == 200:
                    with open(chemin_csv_local, 'wb') as fichier:
                        fichier.write(reponse.content)
                    print(f"Fichier CSV téléchargé à : {chemin_csv_local}")
                else:
                    print(f"Échec du téléchargement du fichier CSV. Code d'état : {reponse.status_code}")
                    
            elif  "excel" in url_:
                chemin_excel_local = os.path.join(dossier_extraction, f"fichier{dep_}.xlsx")
                reponse = requests.get(url_,verify=False)
                if reponse.status_code == 200:
                    with open(chemin_excel_local, 'wb') as fichier:
                        fichier.write(reponse.content)
                    print(f"Fichier Excel téléchargé à : {chemin_excel_local}")
                else:
                    print(f"Échec du téléchargement du fichier Excel. Code d'état : {reponse.status_code}")
            else:
              
                chemin_zip_local = os.path.join(dossier_telechargement, nom_fichier)
                time.sleep(10)
                reponse = requests.get(url_,verify=False)
                if reponse.status_code == 200:
                
                    with open(chemin_zip_local, 'wb') as fichier:
                        fichier.write(reponse.content)

                    print(f"Fichier ZIP téléchargé à : {chemin_zip_local}")

                 
                    with zipfile.ZipFile(chemin_zip_local, 'r') as zip_ref:
                        zip_ref.extractall(dossier_extraction)

                    print(f"Fichiers extraits dans : {dossier_extraction}")
                else:
                    print(f"Échec du téléchargement du fichier ZIP. Code d'état : {reponse.status_code}")

        except Exception as e:
            print(f"Une erreur s'est produite : {e}")

In [10]:
# Exemple d'utilisation 
nom_dossier = "site_et_sols_pollues"
teléchargement(liste_departements,site_et_sols_pollues,nom_dossier,path_2)

C:\Users\ezano\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'files.georisques.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Fichier ZIP téléchargé à : C:\Users\ezano\Documents\GitHub\georisques_data\telechargements\tri_test\tri_2020_sig_di_02.zip
Fichiers extraits dans : C:\Users\ezano\Documents\GitHub\georisques_data\tri_test\02


C:\Users\ezano\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'files.georisques.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Fichier ZIP téléchargé à : C:\Users\ezano\Documents\GitHub\georisques_data\telechargements\tri_test\tri_2020_sig_di_03.zip
Fichiers extraits dans : C:\Users\ezano\Documents\GitHub\georisques_data\tri_test\03


# Création fichiers csv et yaml : risque par risque 

##  fichiers types tri_2020 : Création dataFrame et sauvegarde  au format yaml 

In [11]:
# filtre sur les differentes catégories 
def determine_type(filename):
    if "_s_" in filename:
        return "s"
    else:
        return "Non trouve"

def extraire_informations(type_, filename):
    # Extraire la ville en utilisant une expression régulière
    ville_match = re.search(r'n_tri_(.*?)_', filename)
    if ville_match:
        ville = ville_match.group(1)
    else:
        ville = "Non trouve"  # Si le motif n'est pas trouvé, mettez "Non trouvé" par défaut
    
    # Vérifier la catégorie
    categorie = "autre"  # Par défaut, la catégorie est "autre"
    for cat_mot in ['commune', 'inondable', 'iso_ht','enjeu_rapport']:
        if cat_mot in filename:
            categorie = cat_mot
    
    # Extraire le type_commune en utilisant une expression régulière
    type_commune_match = re.search(fr'{categorie}_(.*?)_{type_}_', filename)
    if type_commune_match:
        type_commune = type_commune_match.group(1).split('_')[1]
    else:
        type_commune = "Non trouve"  # Si le motif n'est pas trouvé, mettez "Non trouvé" par défaut
    
    return ville, categorie, type_commune

In [18]:
# Application

def construction_dataFrame (path_2,nom_dossier):
    # Obtenez le nom de base du chemin
    base_directory = os.path.join(path_2,nom_dossier)
    base_name = os.path.basename(base_directory)

    # Créez une liste vide pour stocker les informations sur les fichiers
    file_info = []

    # Parcourez les fichiers du répertoire
    for root, dirs, files in os.walk(base_directory):
        for file in files:
            # Obtenez le nom complet du fichier
            file_path = os.path.join(root, file)
            # Obtenez le nom du fichier (sans le chemin)
            file_name = os.path.basename(file_path)
            # Divisez le nom du fichier en nom et extension
            name, extension = os.path.splitext(file_name)
            type_fichier = determine_type(name)
            ville, categorie, type_commune = extraire_informations(type_fichier, name)
            # Obtenez le chemin relatif par rapport au répertoire de base
            relative_path = os.path.relpath(file_path, base_directory)
            # Ajoutez le nom de base au début du chemin relatif
            relative_path = os.path.join(base_name, relative_path)
            # Obtenez le nom du sous-dossier
            sous_dossier = os.path.basename(os.path.dirname(file_path))
            # Ajoutez les informations du fichier à la liste
            file_info.append({'Chemin relatif': relative_path, 'Nom': name, 'Extension': extension, 'Sous_dossier': sous_dossier,
                             'Type_surface': type_fichier,
                            'ville': ville,
                             'categorie':categorie,
                             'type_commune':type_commune})
    # Convertir la liste de dictionnaires en format YAML
    yaml_data = yaml.dump(file_info)

    # Enregistrer le fichier YAML
    with open(f'{nom_dossier}.yaml', 'w') as yaml_file:
        yaml_file.write(yaml_data)

    print("Le fichier YAML a été enregistré avec succès.")


    # Créez un DataFrame à partir des informations sur les fichiers
    df = pd.DataFrame(file_info)
    return(df)



In [57]:
nom_dossier = 'tri_2020'
df = construction_dataFrame(path_2,nom_dossier)
df

,Chemin relatif,Nom,Extension,Sous_dossier,Type_surface,ville,categorie,type_commune
0,tri_2020\01\tri_2020_sig_di_01\FRD_TRI_LYON\n_...,n_tri_lyon_commune_s_001,.dbf,FRD_TRI_LYON,s,lyon,commune,Non trouve
1,tri_2020\01\tri_2020_sig_di_01\FRD_TRI_LYON\n_...,n_tri_lyon_commune_s_001,.prj,FRD_TRI_LYON,s,lyon,commune,Non trouve
2,tri_2020\01\tri_2020_sig_di_01\FRD_TRI_LYON\n_...,n_tri_lyon_commune_s_001,.qix,FRD_TRI_LYON,s,lyon,commune,Non trouve
3,tri_2020\01\tri_2020_sig_di_01\FRD_TRI_LYON\n_...,n_tri_lyon_commune_s_001,.shp,FRD_TRI_LYON,s,lyon,commune,Non trouve
4,tri_2020\01\tri_2020_sig_di_01\FRD_TRI_LYON\n_...,n_tri_lyon_commune_s_001,.shx,FRD_TRI_LYON,s,lyon,commune,Non trouve
...,...,...,...,...,...,...,...,...
6191,tri_2020\976\tri_2020_sig_di_976\FRM_TRI_MAYOT...,n_tri_mayotte_iso_ht_03_04fai_s_976,.dbf,FRM_TRI_MAYOTTE,s,mayotte,iso_ht,04fai
6192,tri_2020\976\tri_2020_sig_di_976\FRM_TRI_MAYOT...,n_tri_mayotte_iso_ht_03_04fai_s_976,.prj,FRM_TRI_MAYOTTE,s,mayotte,iso_ht,04fai
6193,tri_2020\976\tri_2020_sig_di_976\FRM_TRI_MAYOT...,n_tri_mayotte_iso_ht_03_04fai_s_976,.qix,FRM_TRI_MAYOTTE,s,mayotte,iso_ht,04fai
6194,tri_2020\976\tri_2020_sig_di_976\FRM_TRI_MAYOT...,n_tri_mayotte_iso_ht_03_04fai_s_976,.shp,FRM_TRI_MAYOTTE,s,mayotte,iso_ht,04fai


# Remontee de nappes : Création dataFrame et sauvegarde  au format yaml 

In [23]:
def name_(filename): 
    if "IMPERMEABLE" in filename and "BDLISA" in filename:
        type_ = "BDLISA"
        caracteristique = "IMPERMEABLE"
    elif "Re_Nappe_fr" in filename and "fiabilite" in filename:
        type_ = "Re_Nappe_fr"
        caracteristique = "fiabilite"
    elif "Re_Nappe_fr" in filename:
        type_ = "Re_Nappe_fr"
        caracteristique = "Non trouve"
    elif "Rexite" in filename and "vs_fiabilite" in filename:
        type_ = "Re_Nappe_fr"
        caracteristique = "Rexite_vs_fiabilite"
    else: 
        type_ = "autre"
        caracteristique = "autre"
        
    return type_, caracteristique

In [24]:


def construction_dataFrame_ (path_2,nom_dossier):
    # Obtenez le nom de base du chemin
    base_directory = os.path.join(path_2,nom_dossier)
    base_name = os.path.basename(base_directory)

    # Créez une liste vide pour stocker les informations sur les fichiers
    file_info = []

    # Parcourez les fichiers du répertoire
    for root, dirs, files in os.walk(base_directory):
        for file in files:
            # Obtenez le nom complet du fichier
            file_path = os.path.join(root, file)
            # Obtenez le nom du fichier (sans le chemin)
            file_name = os.path.basename(file_path)
            # Divisez le nom du fichier en nom et extension
            name, extension = os.path.splitext(file_name)
            # Obtenez le chemin relatif par rapport au répertoire de base
            relative_path = os.path.relpath(file_path, base_directory)
            # Ajoutez le nom de base au début du chemin relatif
            relative_path = os.path.join(base_name, relative_path)
            type_,caracteristique = name_(relative_path)
            # Obtenez le nom du sous-dossier
            sous_dossier = os.path.basename(os.path.dirname(file_path))
            # Ajoutez les informations du fichier à la liste
            file_info.append({'Chemin relatif': relative_path, 'Nom': name, 'Extension': extension, 'Sous_dossier': sous_dossier,
                             'Type_surface': type_,
                             'caracteristique':caracteristique})
    # Convertir la liste de dictionnaires en format YAML
    yaml_data = yaml.dump(file_info)

    # Enregistrer le fichier YAML
    with open(f'{nom_dossier}.yaml', 'w') as yaml_file:
        yaml_file.write(yaml_data)

    print("Le fichier YAML a été enregistré avec succès.")

    # Créez un DataFrame à partir des informations sur les fichiers
    df = pd.DataFrame(file_info)
    return(df)



In [25]:
nom_dossier = 'remontee_nappe'
df = construction_dataFrame_(path_2,nom_dossier)
df

Le fichier YAML a été enregistré avec succès.


,Chemin relatif,Nom,Extension,Sous_dossier,Type_surface,caracteristique
0,remontee_nappe\Dept_75\Remontee_Nappe_2018_RP_...,Remontee_Nappe_2018_RP_65452,.pdf,Dept_75,autre,autre
1,remontee_nappe\Dept_75\MASQUES\Masque_BDLISA_I...,Masque_BDLISA_IMPERMEABLE,.cpg,MASQUES,BDLISA,IMPERMEABLE
2,remontee_nappe\Dept_75\MASQUES\Masque_BDLISA_I...,Masque_BDLISA_IMPERMEABLE,.dbf,MASQUES,BDLISA,IMPERMEABLE
3,remontee_nappe\Dept_75\MASQUES\Masque_BDLISA_I...,Masque_BDLISA_IMPERMEABLE,.prj,MASQUES,BDLISA,IMPERMEABLE
4,remontee_nappe\Dept_75\MASQUES\Masque_BDLISA_I...,Masque_BDLISA_IMPERMEABLE,.shp,MASQUES,BDLISA,IMPERMEABLE
5,remontee_nappe\Dept_75\MASQUES\Masque_BDLISA_I...,Masque_BDLISA_IMPERMEABLE,.shx,MASQUES,BDLISA,IMPERMEABLE
6,remontee_nappe\Dept_75\SYMBOLOGY\ARCGIS\Masque...,Masque_BDLISA_IMPERMEABLE,.lyr,ARCGIS,BDLISA,IMPERMEABLE
7,remontee_nappe\Dept_75\SYMBOLOGY\ARCGIS\Re_Nap...,Re_Nappe_fr,.lyr,ARCGIS,Re_Nappe_fr,Non trouve
8,remontee_nappe\Dept_75\SYMBOLOGY\ARCGIS\Re_Nap...,Re_Nappe_fr_fiabilite,.lyr,ARCGIS,Re_Nappe_fr,fiabilite
9,remontee_nappe\Dept_75\SYMBOLOGY\QGIS\Masque_B...,Masque_BDLISA_IMPERMEABLE,.qml,QGIS,BDLISA,IMPERMEABLE


#  Autres risques : Création dataFrame et sauvegarde  au format yaml 

In [26]:
def DataFrame_autres_risques (path_2,nom_dossier):
    # Obtenez le nom de base du chemin
    base_directory = os.path.join(path_2,nom_dossier)
    base_name = os.path.basename(base_directory)

    # Créez une liste vide pour stocker les informations sur les fichiers
    file_info = []

    # Parcourez les fichiers du répertoire
    for root, dirs, files in os.walk(base_directory):
        for file in files:
            # Obtenez le nom complet du fichier
            file_path = os.path.join(root, file)
            # Obtenez le nom du fichier (sans le chemin)
            file_name = os.path.basename(file_path)
            # Divisez le nom du fichier en nom et extension
            name, extension = os.path.splitext(file_name)
            # Obtenez le chemin relatif par rapport au répertoire de base
            relative_path = os.path.relpath(file_path, base_directory)
            # Ajoutez le nom de base au début du chemin relatif
            relative_path = os.path.join(base_name, relative_path)
            # Obtenez le nom du sous-dossier
            sous_dossier = os.path.basename(os.path.dirname(file_path))
            # Ajoutez les informations du fichier à la liste
            file_info.append({'Chemin relatif': relative_path, 'Nom': name, 'Extension': extension, 'Sous_dossier': sous_dossier})
    # Convertir la liste de dictionnaires en format YAML
    yaml_data = yaml.dump(file_info)

    # Enregistrer le fichier YAML
    with open(f'{nom_dossier}.yaml', 'w') as yaml_file:
        yaml_file.write(yaml_data)

    print("Le fichier YAML a été enregistré avec succès.")

    # Créez un DataFrame à partir des informations sur les fichiers
    df = pd.DataFrame(file_info)
    return(df)

In [66]:
nom_dossier = 'site_et_sols_pollues'
df = DataFrame_autres_risques(path_2,nom_dossier)
df


,Chemin relatif,Nom,Extension,Sous_dossier
0,site_et_sols_pollues\01\fichier01.xlsx,fichier01,.xlsx,01
1,site_et_sols_pollues\02\fichier02.xlsx,fichier02,.xlsx,02
2,site_et_sols_pollues\03\fichier03.xlsx,fichier03,.xlsx,03
3,site_et_sols_pollues\04\fichier04.xlsx,fichier04,.xlsx,04
4,site_et_sols_pollues\05\fichier05.xlsx,fichier05,.xlsx,05
...,...,...,...,...
94,site_et_sols_pollues\971\fichier971.xlsx,fichier971,.xlsx,971
95,site_et_sols_pollues\972\fichier972.xlsx,fichier972,.xlsx,972
96,site_et_sols_pollues\973\fichier973.xlsx,fichier973,.xlsx,973
97,site_et_sols_pollues\974\fichier974.xlsx,fichier974,.xlsx,974


In [29]:
# supprimer les pdf  dans un sous-dossier 
nom_dossier = "remontee_nappe"

In [ ]:
# Parcourez le répertoire principal
repertoire_principal =os.path.join(path_2,nom_dossier)
for root, dirs, files in os.walk(repertoire_principal):
    for file in files:
        if file.endswith(".pdf"):
            fichier_pdf = os.path.join(root, file)
            os.remove(fichier_pdf)
            print(f"Fichier PDF supprimé : {fichier_pdf}")